In [0]:
import pandas as pd
import glob
import yaml
import os
from datetime import datetime

In [0]:
%run ../0_Config.ipynb

In [0]:
output_directory = os.path.join(app_config['output_dir_path'],"Modeling_Results","Consolidated_Results")

In [0]:
all_files = [file for file in os.listdir(output_directory)]
all_files = [file.replace(".csv","") for file in all_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in all_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in all_files if max_date in x]
comb_rsults_file_path = os.path.join(output_directory,req_file_name[0] + ".csv")
comb_rsults = pd.read_csv(comb_rsults_file_path)
comb_rsults[app_config['date_var']] = pd.to_datetime(comb_rsults[app_config['date_var']])

print("Overall data size",comb_rsults.shape)

/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,20,21,27,28,29,30,59,60,61,62,63,64,70,73,74,75,76,77,78,79,89,124) have mixed types.Specify dtype option on import or set low_memory=False.
 has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Overall data size (6064, 164)

In [0]:
# Filtered the data for the algorithms which we've calculated contributions
comb_rsults1 = comb_rsults[comb_rsults["algorithm"].isin(["Prophet","ElasticNet","XGBoost",'Lasso_cvglmnet'])]
print("required alogorithms data size",comb_rsults1.shape)

# Filling nulls with 0
comb_rsults1 = comb_rsults1.fillna(0)

# Getting independent variables from the config file
temp_idvs = []
all_algos = [algo for algo in app_config['Algorithms'].keys() if algo not in ['DeepAR','DeepState']]
for algo in all_algos:
    if("exogenous_variables" in app_config['Algorithms'][algo].keys()):
        temp_idvs = temp_idvs + (app_config['Algorithms'][algo]['exogenous_variables']['positive_corr']+\
                                 app_config['Algorithms'][algo]['exogenous_variables']['negative_corr']+\
                                 app_config['Algorithms'][algo]['exogenous_variables']['uncertain_corr'])
for algo in ['DeepAR','DeepState']:
    temp_idvs = temp_idvs + (app_config['Algorithms'][algo]['exogenous_variables']['feat_dynamic_real'])
contributed_cols = list(set(temp_idvs))

contributed_cols = [i+"_contribution" for i in contributed_cols]
contributed_cols = contributed_cols + ['base','Intercept','expected_value','holidays','trend','yearly','weekly','daily'] 

#Filtering only the common variables btw the contributed_cols and consolidated results
contributed_cols = [i for i in contributed_cols if i in comb_rsults.columns]

comb_rsults1['total_contributions'] = comb_rsults1[contributed_cols].sum(axis=1)
comb_rsults1['difference_check'] = comb_rsults1['yhat'] - comb_rsults1['total_contributions']

comb_rsults1['difference_check_abs'] = comb_rsults1['difference_check'].abs()

# Filtering for the results where total contributions are not matching with prediction
temp = comb_rsults1[comb_rsults1["difference_check_abs"] > 0]
print("Minimum diff",temp['difference_check_abs'].min())
print("Maximum diff",temp['difference_check_abs'].max())
temp

required alogorithms data size (3000, 164)
Minimum diff 9.71445146547012e-17
Maximum diff 2.842170943040401e-14
Out[37]:

,Div_No,Store_No,Base_UPC,Week_Start_Date,y,yhat,test_flag,stateLevel_grocery_and_pharmacy_percent_change_from_baseline_contribution,AvgUnitPrice_contribution,New_cases_contribution,tpr_discount_contribution,base,mape,wmape,bias,tracking_signal,mae,rmse,alpha,l1_ratio,fit_intercept,positive,median_baseprice_contribution,window,status,algorithm,smoothing_level,optimized,exponential,damped_trend,use_brute,yhat_upper,yhat_lower,trend,yearly,weekly,daily,New Year's Day,Martin Luther King Jr. Day,Washington's Birthday,...,Pre_MemorialDay,Superbowl,MemorialDay,Valentine,Pre_Christmas,Post_MemorialDay,Pre_Halloween,4thJuly,Pre_4thJuly,Post_Superbowl,Pre_Easter,NewYear,Pre_Superbowl,Post_4thJuly,Post_LaborDay,Post_Valentine,Pre_Thanksgiving,Christmas,Post_Halloween,Post_Thanksgiving,Thanksgiving_raw,Pre_LaborDay,Post_NewYear,Pre_Valentine,LaborDay,Post_Easter,Halloween,gran_tempp,granular_trend,higher_level_trend,prophet_si_yearly,prophet_si_weekly,prophet_si_daily,cal_si_weekly,cal_si_monthly,cal_si_qtrly,experiment_id,total_contributions,difference_check,difference_check_abs
16,24,8,2200015934,2018-11-28 00:00:00,9,6.920869,0,0.000000,-0.0,0.0,2.573205,4.347665,73.729527,43.967695,3.120627e-16,2.048282e-14,2.254830,2.774813,0.75,0.1,True,True,0.000000,1.0,success,ElasticNet,0.0,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.482200e+12,4.666519,45.011234,-28.832706,8.886557,11.848756,1.349996,0.686321,0.824623,3114000483440696,6.920869,-8.881784e-16,8.881784e-16
39,24,8,2200015934,2019-05-08 00:00:00,8,7.242520,0,0.000000,-0.0,0.0,2.894855,4.347665,73.729527,43.967695,3.120627e-16,2.048282e-14,2.254830,2.774813,0.75,0.1,True,True,0.000000,1.0,success,ElasticNet,0.0,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.482200e+12,4.891293,45.461665,14.865490,8.886557,11.848756,1.367977,1.310560,1.126771,3114000483440696,7.242520,8.881784e-16,8.881784e-16
54,24,8,2200015934,2019-08-21 00:00:00,8,7.242520,0,0.000000,-0.0,0.0,2.894855,4.347665,73.729527,43.967695,3.120627e-16,2.048282e-14,2.254830,2.774813,0.75,0.1,True,True,0.000000,1.0,success,ElasticNet,0.0,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.482200e+12,5.037884,45.755424,19.810209,8.886557,11.848756,1.333941,1.163332,1.066848,3114000483440696,7.242520,8.881784e-16,8.881784e-16
85,24,8,2200015934,2020-03-25 00:00:00,4,1.442997,0,-2.904668,-0.0,-0.0,0.000000,4.347665,73.729527,43.967695,3.120627e-16,2.048282e-14,2.254830,2.774813,0.75,0.1,True,True,0.000000,1.0,success,ElasticNet,0.0,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.482200e+12,5.340840,46.362527,-3.715400,8.886557,11.848756,1.153532,1.334567,0.981759,3114000483440696,1.442997,2.220446e-16,2.220446e-16
88,24,8,2200015934,2020-04-15 00:00:00,2,2.895331,0,-1.452334,-0.0,-0.0,0.000000,4.347665,73.729527,43.967695,3.120627e-16,2.048282e-14,2.254830,2.774813,0.75,0.1,True,True,0.000000,1.0,success,ElasticNet,0.0,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.482200e+12,5.370158,46.421279,25.700005,8.886557,11.848756,1.353099,1.224009,1.126771,3114000483440696,2.895331,-4.440892e-16,4.440892e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.